# Composable Indices Demo

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [1]:
from gpt_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader
)

#### Load Datasets

Load both the NYC Wikipedia page as well as Paul Graham's "What I Worked On" essay

In [2]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('../test_wiki/data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [2]:
# load NYC dataset
nyc_documents = SimpleDirectoryReader('../test_wiki/data/').load_data()

In [3]:
# load PG's essay
essay_documents = SimpleDirectoryReader('../paul_graham_essay/data/').load_data()

### Building the document indices
Build a tree index for the NYC wiki page and PG essay

In [ ]:
# build NYC index
nyc_index = GPTSimpleVectorIndex(nyc_documents)

In [5]:
nyc_index.save_to_disk('index_nyc.json')

In [ ]:
# build essay index
essay_index = GPTSimpleVectorIndex(essay_documents)

In [7]:
essay_index.save_to_disk('index_pg.json')

### Loading the indices
Build a tree indices for the NYC wiki page and PG essay

In [8]:
# try loading
nyc_index = GPTSimpleVectorIndex.load_from_disk('index_nyc.json')
essay_index = GPTSimpleVectorIndex.load_from_disk('index_pg.json')

### Set summaries for the indices

Add text summaries to indices, so we can compose other indices on top of it

In [9]:
nyc_index.set_text("""
    New York, often called New York City or NYC, 
    is the most populous city in the United States. 
    With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), 
    New York City is also the most densely populated major city in the United States, 
    and is more than twice as populous as second-place Los Angeles. 
    New York City lies at the southern tip of New York State, and 
    constitutes the geographical and demographic center of both the 
    Northeast megalopolis and the New York metropolitan area, the 
    largest metropolitan area in the world by urban landmass.[8] With over 
    20.1 million people in its metropolitan statistical area and 23.5 million 
    in its combined statistical area as of 2020, New York is one of the world's 
    most populous megacities, and over 58 million people live within 250 mi (400 km) of 
    the city. New York City is a global cultural, financial, and media center with 
    a significant influence on commerce, health care and life sciences, entertainment, 
    research, technology, education, politics, tourism, dining, art, fashion, and sports. 
    Home to the headquarters of the United Nations, 
    New York is an important center for international diplomacy,
    an established safe haven for global investors, and is sometimes described as the capital of the world.
""") 
essay_index.set_text("""
    Author: Paul Graham. 
    The author grew up painting and writing essays. 
    He wrote a book on Lisp and did freelance Lisp hacking work to support himself. 
    He also became the de facto studio assistant for Idelle Weber, an early photorealist painter. 
    He eventually had the idea to start a company to put art galleries online, but the idea was unsuccessful. 
    He then had the idea to write software to build online stores, which became the basis for his successful company, Viaweb. 
    After Viaweb was acquired by Yahoo!, the author returned to painting and started writing essays online. 
    He wrote a book of essays, Hackers & Painters, and worked on spam filters. 
    He also bought a building in Cambridge to use as an office. 
    He then had the idea to start Y Combinator, an investment firm that would 
    make a larger number of smaller investments and help founders remain as CEO. 
    He and his partner Jessica Livingston ran Y Combinator and funded a batch of startups twice a year. 
    He also continued to write essays, cook for groups of friends, and explore the concept of invented vs discovered in software. 

""")

### Build Keyword Table Index on top of tree indices! 

We set summaries for each of the NYC and essay indices, and then compose a keyword index on top of it.

In [10]:
# set query config
query_configs = [
    {
        "index_struct_type": "simple_dict",
        "query_mode": "default",
        "query_kwargs": {
            "similarity_top_k": 1
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "simple",
        "query_kwargs": {}
    },
]

In [ ]:
keyword_table = GPTSimpleKeywordTableIndex([nyc_index, essay_index], max_keywords_per_chunk=50)

### Define Graph

In [18]:
from gpt_index.composability import ComposableGraph

In [19]:
graph = ComposableGraph.build_from_index(keyword_table)

In [32]:
# [optional] save to disk
graph.save_to_disk("index_graph.json")

In [33]:
# [optional] load from disk
graph = ComposableGraph.load_from_disk("index_graph.json")

In [ ]:
# set Logging to DEBUG for more detailed outputs
# ask it a question about NYC 
response = graph.query(
    "What is the climate of New York City like? How cold is it during the winter?", 
    query_configs=query_configs
)

In [27]:
print(str(response))



New York City has a humid subtropical climate (Cfa) under the Köppen climate classification. Winters are typically chilly and damp, with temperatures usually dropping to 10 °F (−12 °C) several times per winter, yet can also reach 60 °F (16 °C) for several days even in the coldest winter month. The daily mean temperature in January, the area's coldest month, is 33.3 °F (0.7 °C). The city receives an average of 46.9 inches (1,194 mm) of rainfall annually, with the wettest month being August 2011, with 18.95 inches (481 mm) of rainfall. The snowiest month on record is February 2010, with 36.9 inches (94 cm) of snowfall. The snowiest season (Jul–Jun) on record is 1995–1996, with 75.6 inches (192 cm) of snowfall.


In [28]:
# Get source of response
print(response.get_formatted_sources())

> Source (Doc id: 4e8c9bbc-b42f-479f-8fb1-83d0b6198f1d): 
    New York, often called New York City or NYC, 
    is the most populous city in the United St...

> Source (Doc id: 77f3b3ea-93ab-49c8-b938-2bd3c870a602): has been altered substantially by human intervention, with considerable land reclamation along th...


In [ ]:
# ask it a question about PG's essay
response = graph.query(
    "What did the author do growing up, before his time at Y Combinator?", 
    query_configs=query_configs
)

In [30]:
print(str(response))



The author grew up in England and attended college in the United States. He studied computer science and art, and worked on a variety of projects, including writing essays, hacking, and working on a Lisp interpreter. He also worked on a startup called Viaweb, which was eventually acquired by Yahoo. He also worked on Interleaf, a high-end, special-purpose hardware and software company, and sought out signature styles at RISD. He also lived in a rent-stabilized apartment in New York, and worked on software projects that could be launched as soon as they were done.


In [31]:
# Get source of response
print(response.get_formatted_sources())

> Source (Doc id: ae92ab9a-c6ed-48c3-b333-d459908dec3f): 
    Author: Paul Graham. 
    The author grew up painting and writing essays. 
    He wrote a bo...

> Source (Doc id: a63ad7c4-87f2-42fd-a32b-f682a022af90): get their initial set of customers almost entirely from among their batchmates.

I had not origin...
